In [1]:
from collections import OrderedDict

import torch
import torch.nn as nn
from torchvision import datasets, transforms
import models
from models.modules import get_child_dict
from torchsummary import summary
import utils
import utils.optimizers as optimizers
from models.encoders.autoencoders import DecoderResidualBlock


/home/woreom/miniconda3/envs/few/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Resnet12

In [16]:
class ResNet12Decoder(nn.Module):
    def __init__(self):
        super(ResNet12Decoder, self).__init__()
        configs = [1, 2, 2, 2]
        # self.linear = nn.Linear(in_features=512, out_features=512*3*3)
        self.conv1 = DecoderResidualBlock(hidden_channels=512, output_channels=256, layers=configs[0])
        self.conv2 = DecoderResidualBlock(hidden_channels=256, output_channels=128, layers=configs[1])
        self.conv3 = DecoderResidualBlock(hidden_channels=128, output_channels=64,  layers=configs[2])
        self.conv4 = DecoderResidualBlock(hidden_channels=64,  output_channels=64,  layers=configs[3])

        self.conv5 = nn.Sequential(
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=3, stride=2, padding=7,
                               output_padding=1, bias=False),
        )

        self.gate = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x = x.reshape((x.shape[0], 512, 1, 1))
        print(x.shape)
        # x = self.linear(x)
        # x = x.reshape((x.shape[0],-1,3,3))
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.gate(x)

        return x

In [17]:
model=ResNet12Decoder()
_input = torch.randn((1, 512, 3, 3))

print(f"decoder input: {_input.shape}")

output = model(_input)

print(f"encoder output: {output.shape}")

model=model.cuda()
print(summary(model,(512, 3, 3)))

decoder input: torch.Size([1, 512, 3, 3])
torch.Size([1, 512, 3, 3])
encoder output: torch.Size([1, 3, 84, 84])
torch.Size([2, 512, 3, 3])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1            [-1, 512, 3, 3]           1,024
              ReLU-2            [-1, 512, 3, 3]               0
            Conv2d-3            [-1, 512, 3, 3]       2,359,296
       BatchNorm2d-4            [-1, 512, 3, 3]           1,024
              ReLU-5            [-1, 512, 3, 3]               0
   ConvTranspose2d-6            [-1, 256, 6, 6]       1,179,648
       BatchNorm2d-7            [-1, 512, 3, 3]           1,024
              ReLU-8            [-1, 512, 3, 3]               0
   ConvTranspose2d-9            [-1, 256, 6, 6]         131,072
DecoderResidualLayer-10            [-1, 256, 6, 6]               0
DecoderResidualBlock-11            [-1, 256, 6, 6]               0
      BatchNorm2d-12  

# Model from repo

In [2]:
model = models.make('convnet4', {'bn_args': {'track_running_stats': False, 'n_episode': 4}},
                        'logistic', {'n_way': 5, 'in_dim': 800})

optimizer, lr_scheduler = optimizers.make(
      'adam', model.parameters(), lr= 0.001)

inner_args = {'n_step': 5, 'encoder_lr': 0.01, 'classifier_lr': 0.01, 'first_order': False, 'frozen': ['bn'], 'reset_classifier': False, 'momentum': 0.0, 'weight_decay': 0.0}


model.eval()
model.reset_classifier()
# a dictionary of parameters that will be updated in the inner loop
params = OrderedDict(model.named_parameters())

for name in list(params.keys()):
    if not params[name].requires_grad:
        params.pop(name)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

_input = torch.randn((25,3,84,84)).to(device)

print(f"encoder input: {_input.shape}")

output = model.encoder(_input, get_child_dict(params, 'encoder'))

print(f"encoder output: {output.shape}")

output = model.classifier(output, get_child_dict(params, 'classifier'))

print(f"classifier output: {output.shape}")

print(summary(model.encoder,(3,84,84)))
print(summary(model.classifier,(1, 1600)))

encoder input: torch.Size([25, 3, 84, 84])


: 

In [3]:
model = models.make('convnet4', {'bn_args': {'track_running_stats': False, 'n_episode': 4}},
                        'logistic', {'n_way': 5, 'in_dim': 800})
optimizer, lr_scheduler = optimizers.make(
      'adam', model.parameters(), lr= 0.001)

inner_args = {'n_step': 5, 'encoder_lr': 0.01, 'classifier_lr': 0.01, 'first_order': False, 'frozen': ['bn'], 'reset_classifier': False, 'momentum': 0.0, 'weight_decay': 0.0}

print(utils.compute_n_params(model))
model.eval()

model.reset_classifier()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

_input = torch.randn((4,5,3,84,84)).to(device)

print(f"encoder input: {_input.shape}")

output = model(x_shot=_input, x_query=_input, y_shot=_input, inner_args=[], meta_train=False)

print(f"encoder output: {output.shape}")


print(summary(model,(3,84,84)))

32.9K
encoder input: torch.Size([4, 5, 3, 84, 84])


TypeError: list indices must be integers or slices, not str

# First Decoder

In [81]:
class Shallow_Decoder(torch.nn.Sequential):

    def __init__(self,
                 hidden=64,
                 channels=1,
                 max_pool=False,
                 layers=4,
                 max_pool_factor=1.0):
        core = [nn.ConvTranspose2d(in_channels=channels, out_channels=hidden, kernel_size=3,
                                   stride=2, padding=1,), nn.BatchNorm2d(hidden), nn.ReLU()]
        for _ in range(layers - 2):
            core.extend([nn.ConvTranspose2d(in_channels=hidden, out_channels=hidden, kernel_size=3,
                                            stride=1, padding=0,), nn.BatchNorm2d(hidden), nn.ReLU()])
            
        core.extend([nn.ConvTranspose2d(in_channels=hidden, out_channels=3, kernel_size=2,
                                        stride=1, padding=0,), nn.BatchNorm2d(3),nn.Sigmoid()])
            
        super(Shallow_Decoder, self).__init__(*core)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.reshape((x.shape[0], 1, 40, 40))
        x = super(Shallow_Decoder, self).forward(x)

        return x

In [82]:
model=Shallow_Decoder()
_input = torch.randn((1, 1600))

print(f"decoder input: {_input.shape}")

output = model(_input)

print(f"encoder output: {output.shape}")

model=model.cuda()
print(summary(model,(1600, 1, 1)))

decoder input: torch.Size([1, 1600])
encoder output: torch.Size([1, 3, 84, 84])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [-1, 64, 79, 79]             640
       BatchNorm2d-2           [-1, 64, 79, 79]             128
              ReLU-3           [-1, 64, 79, 79]               0
   ConvTranspose2d-4           [-1, 64, 81, 81]          36,928
       BatchNorm2d-5           [-1, 64, 81, 81]             128
              ReLU-6           [-1, 64, 81, 81]               0
   ConvTranspose2d-7           [-1, 64, 83, 83]          36,928
       BatchNorm2d-8           [-1, 64, 83, 83]             128
              ReLU-9           [-1, 64, 83, 83]               0
  ConvTranspose2d-10            [-1, 3, 84, 84]             771
      BatchNorm2d-11            [-1, 3, 84, 84]               6
          Sigmoid-12            [-1, 3, 84, 84]               0
Total params: 75,657
Tr

# Second Decoder

In [83]:
class Deep_Decoder(torch.nn.Sequential):

    def __init__(self,
                 hidden=64,
                 channels=64,
                 max_pool=False,
                 layers=4,
                 max_pool_factor=1.0):
        core = [nn.ConvTranspose2d(in_channels=channels, out_channels=hidden, kernel_size=3,
                                   stride=2, padding=0,), nn.BatchNorm2d(hidden), nn.ReLU()]
        for _ in range(layers - 2):
            core.extend([nn.ConvTranspose2d(in_channels=hidden, out_channels=hidden, kernel_size=5,
                                            stride=2, padding=2,), nn.BatchNorm2d(hidden), nn.ReLU()])
            
        core.extend([nn.ConvTranspose2d(in_channels=hidden, out_channels=3, kernel_size=6,
                                        stride=2, padding=1,), nn.BatchNorm2d(3),nn.Sigmoid()])
            
        super(Deep_Decoder, self).__init__(*core)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.reshape((x.shape[0], 64, 5, 5))
        # print(x.shape)
        # x = x.expand(x.shape[0], 1600, 3, 3)
        x = super(Deep_Decoder, self).forward(x)

        return x

In [84]:
model=Deep_Decoder()
_input = torch.randn((1, 1600))

print(f"decoder input: {_input.shape}")

output = model(_input)

print(f"encoder output: {output.shape}")

model=model.cuda()
print(summary(model,(1600, 1, 1)))

decoder input: torch.Size([1, 1600])
encoder output: torch.Size([1, 3, 84, 84])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [-1, 64, 11, 11]          36,928
       BatchNorm2d-2           [-1, 64, 11, 11]             128
              ReLU-3           [-1, 64, 11, 11]               0
   ConvTranspose2d-4           [-1, 64, 21, 21]         102,464
       BatchNorm2d-5           [-1, 64, 21, 21]             128
              ReLU-6           [-1, 64, 21, 21]               0
   ConvTranspose2d-7           [-1, 64, 41, 41]         102,464
       BatchNorm2d-8           [-1, 64, 41, 41]             128
              ReLU-9           [-1, 64, 41, 41]               0
  ConvTranspose2d-10            [-1, 3, 84, 84]           6,915
      BatchNorm2d-11            [-1, 3, 84, 84]               6
          Sigmoid-12            [-1, 3, 84, 84]               0
Total params: 249,161
T